In [3]:
import tkinter as tk

class Excel(tk.Frame):
    def __init__(self, master, rows, columns, width , result):
        super().__init__(master)
        l=0
        k=1
        C=["Nom image","Prédiction"]
        for i in range(columns):
            self.make_entry(0, i+1, width, C[i], False,'') 
        for row in range(rows):
            self.make_entry(row+1, 0, 5, row+1, False,'')        
            for column in range(columns):
                self.make_entry(row+1, column+1, width, '', False , result[k][l])
                if(k==0):
                    k=1
                else: 
                    k=0
            l=l+1
        
        
    def make_entry(self, row, column, width, text, state, result):
        e = tk.Entry(self, width=width , 
                     bg ="white", fg ="green"
                     , font = ('times', 11, ' bold '))
        if text: e.insert(0, text)
        if result: e.insert(0, result)
        e['state'] = tk.NORMAL if state else tk.DISABLED
        e.coords = (row-1, column-1)
        e.grid(row=row, column=column)

In [4]:
# importing libraries 
import tkinter as tk 
from tkinter import Message, Text
import cv2 
import os 
import shutil 
import csv 
import numpy as np 
from PIL import Image, ImageTk 
import pandas as pd 
import datetime 
import time 
import tkinter.ttk as ttk 
import tkinter.font as font 
from pathlib import Path 
#---------sklearn importations------------#
from sklearn.svm import SVC
import joblib
#------------tracage des courbes-----------#
import matplotlib.pyplot as plt
#-------------cv2 de openCv -----------#
import cv2
#--------------importation des données----#
import os
#-----------calcule arithmetique---------#
import numpy as np
#-------------Keras---------------------#
from keras.models import Model
from tensorflow import keras
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf


model = tf.keras.models.load_model('MyVggModel .h5',
                custom_objects=None, compile=True, options=None)

svc = joblib.load('BestSVM.pkl')






# ==========================================================================================#

def getTargets():
   

  # ------mettre les noms des classes dans target------#

    target = os.listdir('datasetProjetS5_2020/train')
    target.sort()

  # ------on prend qu'un exemplaire de classes----#

    target_sample = [0] * 38
    for i in range(38):
        target_sample[i] = target[i]

  # --------- retourne exemplaire de classes----#

    return target_sample


# ========================================================================================#

def setElementReady(path):

   # ----declaration----#

    dictionary = []

  # ---objet sift---#

    sift = cv2.SIFT_create()
    image_name = os.listdir(path)
    data = []

  # ---get data---#

    for i in range(len(os.listdir(path))):
        data.append(cv2.imread(path + '/' + image_name[i],
                    cv2.IMREAD_GRAYSCALE))

    indiceNonValide = []
    to = 0
    fr = 0
    to = to + len(os.listdir(path))

    # -----la deuxieme boucle pour passer sur touts les images de la classe i-------#

    for j in range(fr, to):

        # ------------detecter les features de l'image j de la classe i-----#

        BOW = cv2.BOWKMeansTrainer(1)
        (kp, ft) = sift.detectAndCompute(data[j], None)
        if ft is not None:
            BOW.add(ft)
            dictionary.append(BOW.cluster())
        else:
            indiceNonValide.append(j)

  # ---retourne le dictionaire----#

    return (dictionary, indiceNonValide, image_name)


# =======================================================================================================#

def getNumBadSizedImages(x):
    bad = []
    print ('lenght :', len(x))
    for i in range(len(x)):
        if len(x[i][0]) != 128:
            bad.append(i)

    return len(bad)


# ================================================#

def removeBadSizedImages(x, nbreBad):
    for i in range(len(x) - nbreBad - 1, 0, -1):
        if len(x[i][0]) != 128:
            x.pop(i)


# ================================================#

def getX(path):

    # get the form of testing

    (x_test, indiceNonValide, image_name) = setElementReady(path)

    # detect and remove it bad sized images

    nbreBad = getNumBadSizedImages(x_test)
    removeBadSizedImages(x_test, nbreBad)

    # from 3d to 2d

    for i in range(len(x_test)):
        x_test[i] = x_test[i][0]

    # -------------end cleaning xtest ----------------#

    return (x_test, image_name)


# =================================================#

def predire(path, svc):
    (x_test, image_name) = getX(path)
    resultat = svc.predict(x_test)
    target = getTargets()
    resultatFinal = []
    for i in range(len(resultat)):
        resultatFinal.append(target[resultat[i]])
    total_rows = len(resultatFinal)
    resultatFinal = [resultatFinal]
    resultatFinal.append(image_name)
    total_columns = 2

    # app = tk.Tk()

    ex = Excel(window, rows=total_rows, columns=total_columns,
               width=42, result=resultatFinal)
    ex.place(x=240, y=260)
    effacer = tk.Button(
        window,
        text='Effacer',
        fg='white',
        bg='green',
        width=7,
        height=1,
        activebackground='Red',
        font=('times', 11, ' bold '),
        )
    effacer['command'] = lambda idx=ex, binst=effacer: erase(idx, binst)
    effacer.place(x=580, y=500)


# ========================================================================================#

def erase(table, button):
    table.destroy()
    button.destroy()


def erasePrime(table, button, text):
    table.destroy()
    button.destroy()
    text.destroy()


# ========================================================================================#

def getModel():
    svc = joblib.load('BestSVM.pkl')
    print(svc)
    return svc


# ================================================================================#

def getTargetByImage():
    choice = var.get()
    path = txt.get()
    lbl = tk.Label(
        window,
        text='path invalide',
        width=50,
        height=2,
        fg='red',
        bg='white',
        font=('times', 15, ' bold '),
        )
#=======================================================================================#
#=======================================Features========================================#
#=======================================================================================#
    if choice == 2:
        

        if os.path.exists(path):
            lbl.pack_forget()
            if os.path.isfile(path):

                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                img = cv2.GaussianBlur(img, (3, 3), 1)
                x_test = []
                resultat = -1
                sift = cv2.SIFT_create()
                BOW = cv2.BOWKMeansTrainer(1)
                (kp, ft) = sift.detectAndCompute(img, None)
                if ft is not None:
                    BOW.add(ft)
                    x_test = BOW.cluster()
                    resultat = svc.predict(x_test)
                else:
                    print("Cannot get the right shape")

                targets = getTargets()
                

                lbl = tk.Label(
                    window,
                    text=targets[resultat[0]],
                    width=50,
                    height=2,
                    fg='black',
                    bg='white',
                    font=('times', 15, ' bold '),
                    )
                lbl.place(x=100, y=500)

                # ==================================#

                Canevas = tk.Canvas(window)
                img = Image.open(path)
                photo = ImageTk.PhotoImage(img)
                Canevas.config(height=photo.height(),
                               width=photo.width())
                Canevas.create_image(0, 0, anchor=tk.NW, image=photo)
                Canevas.place(x=280, y=250)

                # ==================================#

                effacer = tk.Button(
                    window,
                    text='Effacer',
                    fg='white',
                    bg='green',
                    width=7,
                    height=1,
                    activebackground='Red',
                    font=('times', 11, ' bold '),
                    )
                effacer['command'] = lambda idx=Canevas, binst=effacer, \
                    text=lbl: erasePrime(idx, binst, text)
                effacer.place(x=360, y=540)
                l
            else:
                predire(path, svc)
        else:

            lbl.place(x=180, y=300)
            effacer1 = tk.Button(
                window,
                text='Effacer',
                fg='white',
                bg='green',
                width=7,
                height=1,
                activebackground='Red',
                font=('times', 11, ' bold '),
                )
            effacer1['command'] = lambda idx=lbl, binst=effacer1: \
                erase(idx, binst)
            effacer1.place(x=440, y=400)
            print(effacer1)
#=======================================================================================#
#==========================================VGG==========================================#
#=======================================================================================#            
    elif choice == 1:

        if os.path.exists(path):
            lbl.pack_forget()

            if os.path.isfile(path):
                laclasse = recognizeImage(path, model)
                lbl = tk.Label(
                    window,
                    text=laclasse,
                    width=50,
                    height=2,
                    fg='black',
                    bg='white',
                    font=('times', 15, ' bold '),
                    )
                lbl.place(x=100, y=500)

                # ==================================#

                Canevas = tk.Canvas(window)
                img = Image.open(path)
                photo = ImageTk.PhotoImage(img)
                Canevas.config(height=photo.height(),
                               width=photo.width())
                Canevas.create_image(0, 0, anchor=tk.NW, image=photo)
                Canevas.place(x=280, y=250)

                # ==================================#

                effacer = tk.Button(
                    window,
                    text='Effacer',
                    fg='white',
                    bg='green',
                    width=7,
                    height=1,
                    activebackground='Red',
                    font=('times', 11, ' bold '),
                    )
                effacer['command'] = lambda idx=Canevas, binst=effacer, \
                    text=lbl: erasePrime(idx, binst, text)
                effacer.place(x=360, y=540)
                l
            else:

                
                image_name = os.listdir(path)
                resultatVgg = [0] * len(os.listdir(path))

                  # ---get data---#

                for i in range(len(os.listdir(path))):
                    image_path = path + '/' + image_name[i]
                    resultatVgg[i] = recognizeImage(image_path, model)
                total_rows = len(os.listdir(path))
                resultatVgg = [resultatVgg]
                resultatVgg.append(image_name)
                total_columns = 2
                ex = Excel(window, rows=total_rows,
                           columns=total_columns, width=50,
                           result=resultatVgg)
                ex.place(x=240, y=260)
                effacer = tk.Button(
                    window,
                    text='Effacer',
                    fg='white',
                    bg='green',
                    width=7,
                    height=1,
                    activebackground='Red',
                    font=('times', 11, ' bold '),
                    )
                effacer['command'] = lambda idx=ex, binst=effacer: \
                    erase(idx, binst)
                effacer.place(x=580, y=500)
        else:
            lbl.place(x=180, y=300)
            effacer1 = tk.Button(
                window,
                text='Effacer',
                fg='white',
                bg='green',
                width=7,
                height=1,
                activebackground='Red',
                font=('times', 11, ' bold '),
                )
            effacer1['command'] = lambda idx=lbl, binst=effacer1: \
                erase(idx, binst)
            effacer1.place(x=440, y=400)


# =====================================================================================#

def recognizeImage(image_path, model):
    image = tf.keras.preprocessing.image.load_img(image_path)
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    arr = model.predict(input_arr)[0]
    maxElement = np.amax(arr)
    result = np.where(arr == maxElement)
    result = getNameByNum(result[0][0])
    return result


def getNameByNum(num):
    targets = getTargets()
    return targets[num]


# =======================================================================================#


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:313: UserWarning: Trying to unpickle estimator SVC from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [5]:
#===================créer la fenêtre===============#
window = tk.Tk()
window.title('Leaves_Recogniser')
window.configure(background='white')
window.attributes('-fullscreen', True)
window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)
message = tk.Label(
    window,
    text='Leaves-Recognition-System',
    bg='green',
    fg='white',
    width=60,
    height=3,
    font=('times', 30, 'bold'),
    )
message.place(x=0, y=20)

#=================La partie du path============#

lbl = tk.Label(
    window,
    text='Path :',
    width=20,
    height=2,
    fg='green',
    bg='white',
    font=('times', 15, ' bold '),
    )
lbl.place(x=-2, y=200)

txt = tk.Entry(window, width=70, bg='white', fg='green', font=('times',
               15, ' bold '))
txt.place(x=210, y=217)

#====================Partie prediction==============#

lbl2 = tk.Label(
    window,
    text='Prediction :',
    width=20,
    height=2,
    fg='green',
    bg='white',
    font=('times', 15, ' bold '),
    )
lbl2.place(x=0, y=300)

#===========Radio button Vgg ou features===========#

var = tk.IntVar()
radio1 = tk.Radiobutton(
    window,
    text='Vgg16',
    variable=var,
    value=1,
    width=20,
    height=2,
    fg='green',
    bg='white',
    font=('times', 15, ' bold '),
    )
radio1.place(x=50, y=560)

radio2 = tk.Radiobutton(
    window,
    text='Features',
    variable=var,
    value=2,
    width=20,
    height=2,
    fg='green',
    bg='white',
    font=('times', 15, ' bold '),
    )
radio2.place(x=260, y=560)

#=======les button predict et quitter======#

predict = tk.Button(
    window,
    text='predict',
    command=getTargetByImage,
    fg='white',
    bg='green',
    width=20,
    height=3,
    activebackground='Red',
    font=('times', 15, ' bold '),
    )
predict.place(x=110, y=650)

quitWindow = tk.Button(
    window,
    text='Quit',
    command=window.destroy,
    fg='white',
    bg='green',
    width=20,
    height=3,
    activebackground='Red',
    font=('times', 15, ' bold '),
    )
quitWindow.place(x=410, y=650)

window.mainloop()


lenght : 8


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-4-1f09d9b9e642>", line 284, in getTargetByImage
    l
NameError: name 'l' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-4-1f09d9b9e642>", line 284, in getTargetByImage
    l
NameError: name 'l' is not defined


lenght : 8
